## DJL을 활용한 배포

- DJL은 SageMaker에서 LLM을 배포하는 용도로 만들어진 container image입니다. 기본적으로 DeepSpeed, FasterTransformer를 활용할 수 있습니다.
- LLM을 배포할 때는 일반적인 `HuggingFaceModel` 객체를 사용한 HuggingFace 기본 DLC 보다 DJL을 사용하는 것이 좋습니다. 모델 로딩 시 30GB 가 넘어가면 EBS volume 이 충분함에도 root volume 크기 문제로 제대로 로딩이 안될 수 있기 때문입니다. (Training은 상관없습니다.)
- 공식 문서의 튜토리얼에 잘 설명이 되어 있습니다 : https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-tutorials.html
- DJL을 활용한 배포 예시 : https://github.com/dhawalkp/dolly-12b/blob/main/dolly-12b-deepspeed-sagemaker.ipynb
- DJL의 deepspeed 기본 inference 코드 : https://github.com/deepjavalibrary/djl-serving/blob/master/engines/python/setup/djl_python/deepspeed.py


### Container that used for deployment
- deepspeed: `763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.0-cu117`
- fastertransformer: `763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.21.0-fastertransformer5.3.0-cu117`

In [ ]:
%store -r

In [ ]:
model_artifact

In [ ]:
import boto3
import sagemaker
from sagemaker.utils import name_from_base
from sagemaker import image_uris

### DJL container 이미지 선택

- 미리 준비된 DJL 이미지를 선택합니다. [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html)에서 사용 가능한 이미지 리스트와 어떤 라이브러리 버전을 활용하고 있는지 확인이 가능합니다.

In [ ]:
llm_engine = "deepspeed"
# llm_engine = "fastertransformer"

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
sm_client = sagemaker_session.sagemaker_client
sm_runtime_client = sagemaker_session.sagemaker_runtime_client

In [ ]:
framework_name = f"djl-{llm_engine}"
inference_image_uri = image_uris.retrieve(
    framework=framework_name, region=sagemaker_session.boto_session.region_name, version="0.21.0"
)

print(f"Inference container uri: {inference_image_uri}")

### DJL inference 코드 작성

DJL 사용은 간단합니다. 실제 inference를 위한 `model.py` 파일 (기본 inference script 사용 시 이것도 필요없습니다.) 과 `serving.properties` 파일만 작성해 주면 됩니다. 구조는 복잡하지 않기 때문에 `dolly-src` 에 있는 예시를 참고해 주세요.
- 옵션에 관련해서는 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html) 를 참고해 주세요.
- 아래와 같은 구조를 갖게 됩니다. 해당 디렉토리를 tar.gz 압축 후 s3에 업로드하도록 합니다.

```
- dolly-src
  - instruct_pipeline.py
  - model.py
  - serving.properties
```


### 변경 필요한 부분

- 업로드 된 모델의 s3 주소 (`model_artifact`) 에 맞추어서 `serving.properties` 의 `option.s3url`을 수정해 주어야 합니다.


In [ ]:
s3_target = f"s3://{sagemaker_session.default_bucket()}/llm/databricks/dolly-v2-7b/code/"
print(s3_target)

In [ ]:
!rm dolly-src.tar.gz
!tar zcvf dolly-src.tar.gz dolly-src --exclude ".ipynb_checkpoints" --exclude "__pycache__"
!aws s3 cp dolly-src.tar.gz {s3_target}

In [ ]:
model_uri = f"{s3_target}dolly-src.tar.gz"
print(model_uri)

### 모델 배포

- container image 는 DJL을 사용할 것이고, 코드를 압축해서 s3에 올렸기 때문에 이제 모델을 배포할 차례입니다.
- `model 생성 -> endpoint config 생성 -> endpoint 생성` 순서로 진행하면 되며, 해당 과정은 SageMaker console에서 모두 확인이 가능합니다.

In [ ]:
model_name = name_from_base(f"dolly-7b-djl")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": model_uri},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
# instance_type = "ml.g5.4xlarge"
instance_type = "ml.g4dn.2xlarge"  # dolly-7b 배포시 최소 사양
# instance_type = "ml.g5.12xlarge"  # multi-gpu 가 필요한 모델의 경우

endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ContainerStartupHealthCheckTimeoutInSeconds": 600,
        },
    ],
)
print(endpoint_config_response)

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

### 배포 시작

- Endpoint 생성 요청을 하면 EC2를 할당받고, s3에서 모델을 다운받아서 준비상태가 됩니다.
- 배포는 10분 이상 걸릴 수 있습니다.
- `Status`가 `InService` 가 되면 정상적으로 배포가 된 상태입니다.

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

In [ ]:
import json

In [ ]:
prompt = "Explain to me how to use aws serverless services"

In [ ]:
%%time
prompts = [prompt]
response_model = sm_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "text": prompts,
        }
    ),
    ContentType="application/json",
)

In [ ]:
output = str(response_model["Body"].read(), "utf-8")

In [ ]:
result = json.loads(output)[0][0]["generated_text"]

In [ ]:
print(result)

### 결과

- 텍스트 생성이 잘 동작하는 걸 확인할 수 있습니다.
- SageMaker Endpoint는 여러 기능 (Load balancing, Autoscaling, ...) 이 있기 때문에 이를 활용하여 다양한 LLM 모델을 쉽게 배포할 수 있습니다.